In [33]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import statsmodels.api as stm
import seaborn as sns
%matplotlib qt5

In [34]:
print (os.getcwd())
currDir = os.getcwd()
fullPath = currDir + "/CabaneASucrev0r3.csv"
type(currDir)

/home/hadoop/Git_Ia/CegepSteFoy_IA/04_Eda/TP02


str

In [35]:
###### DataPrep
### lire, fill les nulls
donnee = pd.read_csv(fullPath)
donnee.fillna(method='bfill', inplace=True)

## supprimer los colonnes pixels
dfSansPixel = donnee.drop(donnee.filter(like='Pixel').columns, axis=1)

## créer un df numérique
dfNumerique = dfSansPixel.select_dtypes(exclude='object').copy()
dfNumerique.describe().apply(lambda x: round(x, 2))

## convertir classe sirop en numérique
colsCategSirop = pd.get_dummies(dfSansPixel['Classe Sirop'], prefix='categSirop_', drop_first=True)

## concatenate side by side
dfNumerique = pd.concat([dfNumerique, colsCategSirop], axis=1)

## enelever les données où débit nul de sève
dfNumerique = dfNumerique.loc[dfNumerique['Débit sève (L/j)'] !=0]

In [36]:
### Séparation colonnes X et Y


### liste de variables (colonnes) dépendantes
list_cols_dependennt_vars = []
col_debit_seve = dfSansPixel['Débit sève (L/j)'].name
col_sucre_dans_seve = dfSansPixel['Sucre sève (%)'].name
col_pct_transmittance = dfSansPixel['Transmittance produit (%)'].name
col_productivite_seve_par_saison = dfSansPixel['Production moyenne par entaille (L)'].name
list_cols_dependennt_vars = [col_debit_seve, col_sucre_dans_seve, col_pct_transmittance, col_productivite_seve_par_saison]


## seulement les variables X
dfNumerique_X_cols = dfNumerique.loc[:, ~dfNumerique.columns.isin(list_cols_dependennt_vars)].copy()

lstColTemp = ['Temp max.(°C)', 'Temp min.(°C)']
dfNumerique_X_cols = dfNumerique_X_cols.loc[:,~dfNumerique_X_cols.columns.isin(lstColTemp) ].copy()

## seulement les variables Y
dfNumerique_Y_cols = dfNumerique.loc[:, dfNumerique.columns.isin(list_cols_dependennt_vars)].copy()


#### Après coup, on a appris que ces 2 variables causaient un prob de haute colinéarité.
#### et si on les enlève plus tôt?

lstColsExlude = ['Précip. Tot. Hiver (mm)', 'Temp moy.(°C)']
dfNumerique_X_cols = dfNumerique_X_cols.loc[:, ~dfNumerique_X_cols.columns.isin([col for col in lstColsExlude])]



In [37]:
### obtenir moyenne d'une colonne
def getMeanListExclusions(pColName, pData=dfNumerique, pListeExclusion=[2014, 2015, 2016]):
    return pData[pColName][~pData['Année'].isin(pListeExclusion)].mean()

In [38]:
### obtenir moyenne d'une colonne
def getMeanListInclusions(pColName, pAnnee, pData=dfNumerique ):
    return pData[pColName][pData['Année'] == pAnnee].mean()

In [39]:
#moyenne_transmit_toutes_annees = dfNumerique['Transmittance produit (%)'].mean()
moyenne_transmit_toutes_annees = getMeanListExclusions('Transmittance produit (%)')
moyenne_sucre_toutes_annees = getMeanListExclusions('Sucre sève (%)')

In [40]:
moyenne_transmit_toutes_annees

43.60843108504399

In [41]:
moyenne_sucre_toutes_annees

2.433225806451613

In [42]:
### for these dictionnaries, key = year [2014-2016]
lstAnnees = [2014, 2015, 2016]
dictAnneeTransmittance = {}
dictAnneeSucre = {}
for annee in lstAnnees:    
    dictAnneeTransmittance.update({annee: getMeanListInclusions('Transmittance produit (%)', annee)})
    dictAnneeSucre.update({annee: getMeanListInclusions('Sucre sève (%)', annee)})

In [43]:
### Imnprime la différence entre la moyenne historique (excluant 2014-2016) versus [2014-2016]
def compare_means(pMean1, pMean2, pAnnee, pCol):
    diff = pMean1 - pMean2
    print("-----")
    print("Différence entre moyennes pour ", pCol)
    print("moyenne historique: ", pMean1)
    print("moyenne ", pAnnee)
    print("Différence absolue: ", abs(diff))
    print("Différence %: ", abs(100*(diff/pMean1)))
    

In [44]:
### Transmittance
for key in dictAnneeTransmittance.keys():
    compare_means(moyenne_transmit_toutes_annees, dictAnneeTransmittance.get(key), key, 'Transmittance produit (%)')

-----
Différence entre moyennes pour  Transmittance produit (%)
moyenne historique:  43.60843108504399
moyenne  2014
Différence absolue:  0.8062880160796055
Différence %:  1.8489269070634629
-----
Différence entre moyennes pour  Transmittance produit (%)
moyenne historique:  43.60843108504399
moyenne  2015
Différence absolue:  3.311315700428608
Différence %:  7.5932924391867465
-----
Différence entre moyennes pour  Transmittance produit (%)
moyenne historique:  43.60843108504399
moyenne  2016
Différence absolue:  0.6851244705115676
Différence %:  1.5710825944998026


In [45]:
### Transmittance
for key in dictAnneeSucre.keys():
    compare_means(moyenne_sucre_toutes_annees, dictAnneeSucre.get(key), key, 'Sucre sève (%)')

-----
Différence entre moyennes pour  Sucre sève (%)
moyenne historique:  2.433225806451613
moyenne  2014
Différence absolue:  0.2832258064516129
Différence %:  11.639931061911705
-----
Différence entre moyennes pour  Sucre sève (%)
moyenne historique:  2.433225806451613
moyenne  2015
Différence absolue:  0.34841811414392065
Différence %:  14.319185388388625
-----
Différence entre moyennes pour  Sucre sève (%)
moyenne historique:  2.433225806451613
moyenne  2016
Différence absolue:  0.20099641577060945
Différence %:  8.260491699441726


In [46]:
## https://towardsdatascience.com/demystifying-hypothesis-testing-with-simple-python-examples-4997ad3c5294
def print_analyse_hypothese(pColYName, pMoyenneHistoriqueY, pDict, pDf=dfNumerique, pConf=0.05):
    ## Transmittance produit (%)
    n =len(pDf)
    pnull = pMoyenneHistoriqueY ##0.43732148760330574 #moyenne hystorique
    ### Transmittance
    print("==================================")
    print("var [pColYName]: ", pColYName)
    for key in pDict.keys():
        phat = pDict.get(key) #moyenne de l'année
        print("-------------------")
        print("année = ", key)
        print("n = ", n)
        print("moyenne historique [pnull] = ", pnull)
        print("moyenne de l'année [phat] = ", phat)
        #sm.stats.proportions_ztest(phat * n, n, pnull, alternative='larger')
        zstat, p_value = stm.stats.ztest(pDf[pColYName][pDf['Année']==key], value = pnull ,alternative='two-sided')        
        #zstat, p_value = stm.stats.proportions_ztest( nobs=n, pnull,  alternative='two-sided')
        print("zstat: ", zstat)
        print("p_value: ", p_value)
        print("h0: moyenne historique = moyenne [2014-2016]")
        print("h1: moyenne historique != moyenne [2014-2016]")
        if (p_value < pConf):
            print("p_value < confidence --> on REJÈTE l'hypothèse null ")
        else:
            print("p_value > confidence --> on accèpete l'hypothèse null")

In [47]:
### On appelle la foction avec chacune de variables pour obtenir l'analyse

print_analyse_hypothese('Transmittance produit (%)', moyenne_transmit_toutes_annees, dictAnneeTransmittance)
print_analyse_hypothese('Sucre sève (%)', moyenne_sucre_toutes_annees, dictAnneeSucre)

var [pColYName]:  Transmittance produit (%)
-------------------
année =  2014
n =  1595
moyenne historique [pnull] =  43.60843108504399
moyenne de l'année [phat] =  44.414719101123595
zstat:  0.6790896790540749
p_value:  0.4970810415765258
h0: moyenne historique = moyenne [2014-2016]
h1: moyenne historique != moyenne [2014-2016]
p_value > confidence --> on accèpete l'hypothèse null
-------------------
année =  2015
n =  1595
moyenne historique [pnull] =  43.60843108504399
moyenne de l'année [phat] =  40.29711538461538
zstat:  -2.265067078870803
p_value:  0.023508563484969776
h0: moyenne historique = moyenne [2014-2016]
h1: moyenne historique != moyenne [2014-2016]
p_value < confidence --> on REJÈTE l'hypothèse null 
-------------------
année =  2016
n =  1595
moyenne historique [pnull] =  43.60843108504399
moyenne de l'année [phat] =  44.29355555555556
zstat:  0.5539613071818064
p_value:  0.5796053264135708
h0: moyenne historique = moyenne [2014-2016]
h1: moyenne historique != moyenne 

In [48]:
sns.scatterplot(data = dfNumerique
                , x = dfNumerique['Année']
                , y = dfNumerique['Transmittance produit (%)']
           )

<AxesSubplot: xlabel='Année', ylabel='Transmittance produit (%)'>

In [49]:
sns.relplot(data = dfNumerique, x="Année", y="Transmittance produit (%)", hue="Année")

In [50]:
sns.boxenplot(data = dfNumerique, x="Année", y="Transmittance produit (%)")

<AxesSubplot: xlabel='Année', ylabel='Transmittance produit (%)'>

In [51]:
sns.boxenplot(data = dfNumerique, x="Année", y="Sucre sève (%)")

<AxesSubplot: xlabel='Année', ylabel='Sucre sève (%)'>